In [2]:
import random, glob
import numpy as np

In [3]:
music_files = [a for a in glob.glob("dataset/*/*")]
print("A random song", random.sample(music_files, 1))

A random song ['dataset\\schumann\\scn16_5.mid']


In [4]:
from music21 import midi
def play_midi_file(midi_file_name):
    mf = midi.MidiFile()

    mf.open(midi_file_name) # path='abc.midi'
    mf.read()
    mf.close()
    s = midi.translate.midiFileToStream(mf)
    s.show('midi')

In [5]:
len(music_files)

295

In [6]:
from music21 import converter,corpus, chord

In [7]:
one_sample = music_files[193]
one_sample

'dataset\\mendelssohn\\mendel_op30_2.mid'

In [8]:
play_midi_file(one_sample)

c:\Users\Dell\anaconda3\lib\site-packages\music21\midi\translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2010 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(


In [9]:
tonic_mode_all = []
for i, name in enumerate(music_files[:5]):
    try:
        parsed_score = converter.parse(name)
        key_of_one = parsed_score.analyze('key')
        tonic_mode_all.append((i, name, key_of_one.tonic.name, key_of_one.mode))
    except:
        print(f'Error at {i}')
        tonic_mode_all.append((i, name, 'NAN', 'NAN'))
        continue
    print(f'{tonic_mode_all[i]}')

(0, 'dataset\\albeniz\\alb_esp1.mid', 'D', 'minor')
(1, 'dataset\\albeniz\\alb_esp2.mid', 'D', 'major')
(2, 'dataset\\albeniz\\alb_esp3.mid', 'E', 'minor')
(3, 'dataset\\albeniz\\alb_esp4.mid', 'G', 'minor')
(4, 'dataset\\albeniz\\alb_esp5.mid', 'E-', 'major')


In [10]:
def get_chords(music_files):
    chords = []
    for file_no, filename in enumerate(music_files):
        chords.append(converter.parse(filename).chordify())
        print(f'Happening of {file_no}')
    
    return chords

In [11]:
music_files_25 = music_files[:25]
#get_chords(music_files[:10])

In [12]:
chords_of_all_music = get_chords(music_files_25)

Happening of 0
Happening of 1
Happening of 2
Happening of 3
Happening of 4
Happening of 5
Happening of 6
Happening of 7
Happening of 8
Happening of 9
Happening of 10
Happening of 11
Happening of 12
Happening of 13
Happening of 14
Happening of 15
Happening of 16
Happening of 17
Happening of 18
Happening of 19
Happening of 20
Happening of 21
Happening of 22
Happening of 23
Happening of 24


In [13]:
#compressed chords if all music
chords_of_all_music[:5]

[<music21.stream.Part 0x1fbdbb46550>,
 <music21.stream.Part 0x1fbdb278160>,
 <music21.stream.Part 0x1fbdf6b5760>,
 <music21.stream.Part 0x1fbdad01f40>,
 <music21.stream.Part 0x1fbe07ea400>]

In [14]:
from music21 import chord, duration

Classifying the music into different modes
- firstly, let's code for major mode

In [54]:
from music21 import *

In [61]:
def get_music_midi_from_chords_and_duration(input_chords):
    midi_stream = stream.Stream()

    for note_pattern, duration_pattern in input_chords:
        notes_in_chord = note_pattern.split('.')
        
        chord_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(current_note)
            new_note.duration = duration.Duration(duration_pattern)
            new_note.storedInstrument = instrument.Violoncello()
            chord_notes.append(new_note)
        new_chord = chord.Chord(chord_notes)
        
        midi_stream.append(new_chord)

        new_tempo = tempo.MetronomeMark(number=50)
            
        midi_stream.append(new_tempo)

    midi_stream = midi_stream.chordify()
    timestr = time.strftime("%Y%m%d-%H%M%S")
    new_file = 'output-' + timestr + '.mid'
    return midi_stream.write('midi', fp=new_file)

In [63]:
def get_music_midi_filename_from_chords(input_chords):
    midi_stream = stream.Stream()

    for note_pattern, duration_pattern in input_chords:
        notes_in_chord = note_pattern.split('.')
        
        chord_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(current_note)
            new_note.duration = duration.Duration(duration_pattern)
            new_note.storedInstrument = instrument.Violoncello()
            chord_notes.append(new_note)
        new_chord = chord.Chord(chord_notes)
        
        midi_stream.append(new_chord)

        new_tempo = tempo.MetronomeMark(number=50)
            
        midi_stream.append(new_tempo)

    midi_stream = midi_stream.chordify()
    timestr = time.strftime("%Y%m%d-%H%M%S")
    new_file = 'output-' + timestr + '.mid'
    return midi_stream.write('midi', fp=new_file
)

Generating music from our processed chords...


NameError: name 'processed_chords' is not defined

In [16]:
def round_chord_durations(number):
    if(number>1.25):
        return 2
    if(number>.30):
        return 1.25
    if(number>.10):
        return 0.3
    return 0.1

For chord and duration of a single song(music file)

In [17]:
def get_chord_and_duration_data(indv_midi_chordified):
    chord_duration_data = []
    for element in indv_midi_chordified.flat:
        if isinstance(element, chord.Chord):
            chord_duration = round_chord_durations(element.duration.quarterLength)
            chord_name = ".".join([n.nameWithOctave for n in element.pitches])
            chord_duration_data.append((chord_name, chord_duration))

    return chord_duration_data

In [18]:
chords_and_duration_data_all_music= []
for one_music_chords in chords_of_all_music:
   chords_and_duration_data_all_music.append((get_chord_and_duration_data(one_music_chords)))

In [77]:
# this is the main data:
type(chords_and_duration_data_all_music[0][0][1])

float

In [65]:
print("Generating music from our processed chords...")
proccessed_chords_to_midi_sample = get_music_midi_filename_from_chords(chords_and_duration_data_all_music[0][4:50])
print(proccessed_chords_to_midi_sample)
play_midi_file(proccessed_chords_to_midi_sample)

Generating music from our processed chords...
output-20230129-222251.mid


from Tonic_mode_all, separating major and minor songs 

In [20]:
tonic_mode_all[0][0], tonic_mode_all[0][-1]

(0, 'minor')

In [22]:
all_major_songs_chords_and_duration = []
all_minor_songs_chords_and_duration = []
for i in range(len(tonic_mode_all)):
    if(tonic_mode_all[i][-1] == 'major'):
        try:
            all_major_songs_chords_and_duration.append(chords_and_duration_data_all_music[i])
        except:
            all_major_songs_chords_and_duration.append('NANNNNNN')
    else:
        try:
            all_minor_songs_chords_and_duration.append(chords_and_duration_data_all_music[i])
        except:
             all_minor_songs_chords_and_duration.append('NANNNNNN')

In [23]:
len(all_minor_songs_chords_and_duration), #minor_songs_chords_with_duration, len(minor_songs_chords_with_duration)

(3,)

In [24]:
major_dataset = all_major_songs_chords_and_duration 
minor_dataset = all_minor_songs_chords_and_duration

In [25]:
#just a reference code
#major_dataset
#y=np.array(major_dataset)
#unique = set(major_dataset_all)
#unique2 = set(minor_dataset_all)

In [26]:
chords_and_duration_data_all_music

[[('A3.A5', 1.25),
  ('E4.E6', 2),
  ('E4.E6', 1.25),
  ('D4.E4.D6.E6', 0.3),
  ('D4.D6', 1.25),
  ('C4.C6', 1.25),
  ('D4.D6', 1.25),
  ('E4.E6', 1.25),
  ('F4.F6', 1.25),
  ('G4.G6', 1.25),
  ('E4.E6', 1.25),
  ('F4.F6', 1.25),
  ('E4.E6', 0.3),
  ('D4.E4.D6.E6', 0.3),
  ('D4.D6', 1.25),
  ('C4.C6', 1.25),
  ('B-3.B-5', 1.25),
  ('A2', 0.3),
  ('A2', 0.3),
  ('E3.A3', 1.25),
  ('A3.C#4', 1.25),
  ('C#4.E4', 1.25),
  ('E4.A4', 1.25),
  ('A4.C#5', 1.25),
  ('C#5.E5', 1.25),
  ('C#4.A4', 1.25),
  ('E4.C#5', 1.25),
  ('A4.E5', 1.25),
  ('C#5.A5', 1.25),
  ('E5.C#6', 1.25),
  ('A5.E6', 1.25),
  ('E5.C#6', 1.25),
  ('C#5.A5', 1.25),
  ('A4.E5', 1.25),
  ('E4.C#5', 1.25),
  ('C#4.A4', 1.25),
  ('A4.E5', 1.25),
  ('E4.C#5', 1.25),
  ('C#4.A4', 1.25),
  ('A3.E4', 1.25),
  ('A3.C#4', 1.25),
  ('E3.A3', 1.25),
  ('A2', 1.25),
  ('A2', 0.3),
  ('A3.A5', 1.25),
  ('E4.E6', 2),
  ('E4.E6', 1.25),
  ('D4.E4.D6.E6', 0.3),
  ('D4.D6', 1.25),
  ('C4.C6', 1.25),
  ('D4.D6', 1.25),
  ('E4.E6', 1.25),
  

* Data preparation stage

In [27]:
no_of_timesteps = 32
x = []
y = []
## CD stands for chord and duration.

for each_music_with_CD in chords_and_duration_data_all_music:
    for each_CD in range(0, len(each_music_with_CD) - no_of_timesteps,  1):
        
        ## preparing input and output sequences:
        input_ = each_music_with_CD[i:i + no_of_timesteps]
        output = each_music_with_CD[i + no_of_timesteps]
        
        for i in range(len(input_)):
            input_[i]= "@".join(map(str,input_[i]))
            
        output = "@".join(map(str,output))
        
        x.append(input_)
        y.append(output)
        
x=np.array(x)
y=np.array(y)

In [28]:
y

array(['A4.E5@1.25', 'C4.C6@1.25', 'C4.C6@1.25', ..., 'F#4.G#4@0.3',
       'F#4.G#4@0.3', 'F#4.G#4@0.3'], dtype='<U24')

In [30]:
#assigning unique integer to every chords_and_duration

unique_x_CD = list(set(x.ravel()))
unique_x_CD_to_int = dict((chord_and_duration, number) for number, chord_and_duration in enumerate(unique_x_CD))
unique_x_CD_to_int

{'G4.B-5.D6@1.25': 0,
 'F#2.F#3.B-3.D4@0.3': 1,
 'B-3@0.3': 2,
 'C4.E4.G4@0.3': 3,
 'G2.E3.G3.B3.E4@0.3': 4,
 'C#3.G#3.B3.B4@1.25': 5,
 'C4.G#4@0.3': 6,
 'C#4.G5@0.1': 7,
 'E-2.B-2.G3.C4@1.25': 8,
 'A4.F#5@1.25': 9,
 'E-4.G4.B-4.G5@0.3': 10,
 'C#4@1.25': 11,
 'G#4.B4.F5@0.3': 12,
 'C#4@0.1': 13,
 'G3.B-3.B-4@1.25': 14,
 'B-4.E-5.F#5.B-5@0.1': 15,
 'F2.F#2.C#3.F3.C#4@0.1': 16,
 'A3.A5@1.25': 17,
 'F3.C4.C5.F5.A5.C6@0.1': 18,
 'C#3.F#3.A3.C4.E-4.F#4.A4@1.25': 19,
 'G2.E3.G3.D4.G4.D5@0.3': 20,
 'C#4.E4.G4@0.1': 21,
 'E-5.F#5.E-6@0.3': 22,
 'F3.G#3.C#4.C#5@1.25': 23,
 'C4.G5.F6@0.3': 24,
 'E4.B4.B5@0.3': 25,
 'E-4.E-5@0.3': 26,
 'F3.A5.F6@0.3': 27,
 'E3.B4.C#5.D5.E5@0.3': 28,
 'E4.C#5@1.25': 29,
 'B-2.B-4@1.25': 30,
 'F2.E-3.A3.F4@1.25': 31,
 'C3.G#3.F4.B4@0.3': 32,
 'F3@1.25': 33,
 'C3.E-3.C5@0.3': 34,
 'F#5@0.3': 35,
 'E-3@0.3': 36,
 'G#1.G#2.E-3.F#3.C#4@1.25': 37,
 'F#2.F#3@0.3': 38,
 'C4.F4.E-5.A5@1.25': 39,
 'G#4.A4.C#5.G#5.A5@0.3': 40,
 'B2.F#3.D4.D5@1.25': 41,
 'G2.E3.G3.G4.B4.D5.G5

In [33]:
#preparing input sequences::

x_seq=[]
for each_row in x:
    temp=[]
    for each_piece in each_row:
        #assigning unique integer to every note
        temp.append(unique_x_CD_to_int[each_piece])
    x_seq.append(temp)
    
x_seq = np.array(x_seq)
x_seq.shape

(40407, 32)

In [34]:
# preparing th output sequences as well::

unique_y_CD = list(set(y))
unique_y_CD_to_int = dict((chord_and_duration, number) for number, chord_and_duration in enumerate(unique_y_CD)) 
unique_y_CD_to_int

{'E5@0.3': 0,
 'D3.G3.B3.B5@1.25': 1,
 'G4.C5.E5.B-5.C6@0.3': 2,
 'C4@0.3': 3,
 'C4.G#4@0.3': 4,
 'F4.G#4.B4.D5.E5@0.3': 5,
 'F3.D4.G4.G#4@0.3': 6,
 'C4.C6@1.25': 7,
 'F3@0.3': 8,
 'C#4@0.3': 9,
 'E4.C#5.E5@1.25': 10,
 'B3.D4.F4.G#4.D5@1.25': 11,
 'D5@1.25': 12,
 'F#4@0.3': 13,
 'F3.B-5.D6@0.3': 14,
 'F#4.G#4@0.3': 15,
 'D4@0.3': 16,
 'B-3.D4.F4.B-4@1.25': 17,
 'F#2.G#2.C3.E-3.G#3@1.25': 18,
 'G2.D3.G3.G4.B4.D5.G5@0.3': 19,
 'C4.E4.E5@0.3': 20,
 'A4.E5@1.25': 21,
 'B3.C#5.E5@0.3': 22,
 'F3.F4.F5.A5.F6@0.3': 23,
 'F#3.F4@0.3': 24,
 'G#3.C4.F#4.G#4.G#5@1.25': 25}

In [35]:
y_seq=np.array([unique_y_CD_to_int[i] for i in y])
y_seq.shape

(40407,)

In [36]:
# preserving 80% of the data for training and the rest 20% for the evaluation:

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_seq,y_seq,test_size=0.2,random_state=0)

In [37]:
len(x_val)

8082

# model building phase:

In [38]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *
import keras.backend as K

K.clear_session()
model = Sequential()
    
#embedding layer
model.add(Embedding(len(unique_x_CD), 100, input_length=32,trainable=True)) 

model.add(Conv1D(64,3, padding='causal',activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
    
model.add(Conv1D(128,3,activation='relu',dilation_rate=2,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))

model.add(Conv1D(256,3,activation='relu',dilation_rate=4,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
          
#model.add(Conv1D(256,5,activation='relu'))    
model.add(GlobalMaxPool1D())
    
model.add(Dense(256, activation='relu'))
model.add(Dense(len(unique_y_CD), activation='softmax'))
    
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 100)           53000     
                                                                 
 conv1d (Conv1D)             (None, 32, 64)            19264     
                                                                 
 dropout (Dropout)           (None, 32, 64)            0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 16, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 16, 128)           24704     
                                                                 
 dropout_1 (Dropout)         (None, 16, 128)           0         
                                                        

In [39]:
!pip install h5py

In [40]:
import h5py

In [41]:
#defining call back to save the best model during training>
mc=ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True,verbose=1)

In [42]:
#actual training
history = model.fit(np.array(x_train),np.array(y_train),batch_size=128,epochs=3, 
                   validation_data=(np.array(x_val),np.array(y_val)),verbose=1, callbacks=[mc])

Epoch 1/3
252/253 [============================>.] - ETA: 0s - loss: 0.4291
Epoch 00001: val_loss improved from inf to 0.00367, saving model to best_model.h5
253/253 [==============================] - 13s 45ms/step - loss: 0.4282 - val_loss: 0.0037
Epoch 2/3
252/253 [============================>.] - ETA: 0s - loss: 5.0140e-04
Epoch 00002: val_loss improved from 0.00367 to 0.00116, saving model to best_model.h5
253/253 [==============================] - 12s 47ms/step - loss: 5.0038e-04 - val_loss: 0.0012
Epoch 3/3
253/253 [==============================] - ETA: 0s - loss: 3.0855e-04
Epoch 00003: val_loss improved from 0.00116 to 0.00064, saving model to best_model.h5
253/253 [==============================] - 12s 48ms/step - loss: 3.0855e-04 - val_loss: 6.4479e-04


In [43]:
#loading best model (Previously trained modle)
from keras.models import load_model
model = load_model('best_model.h5')


In [157]:
import numpy as np
import random
ind = np.random.randint(0,len(x_val)-1)
random_music = x_val[ind]
random_music

array([458, 459, 257, 476,  72, 141, 174, 141, 419,  97, 288, 303, 196,
       334, 297, 205,  37, 101, 190, 157,  16, 325, 181, 529, 279, 299,
       210,  60, 189, 509,  46, 161])

In [158]:
print("Generate a prediction")
prediction = model.predict(x_val[0:1])
print("prediction shape:", prediction)
y_pred= np.argmax(prob,axis=0)
y_pred

Generate a prediction
prediction shape: [[5.5624465e-20 2.7517173e-11 1.3762168e-12 4.4509218e-08 2.4125283e-11
  5.7185203e-12 1.2659901e-14 8.9155056e-24 1.1160397e-14 9.6777921e-07
  1.4314086e-18 3.1840299e-17 3.7336681e-19 1.3577591e-13 2.1620157e-11
  9.9999893e-01 3.1981440e-10 1.2080457e-11 1.6124904e-12 7.1885702e-09
  3.0283183e-09 5.1573781e-17 1.5265829e-11 3.9674777e-08 7.5250387e-17
  7.3239548e-08]]


14

In [159]:
predictions=[]
for i in range(10):

    random_music = random_music.reshape(1,32)
    print("random music = ", random_music)
    

    prob  = model.predict(random_music)[0]
    y_pred= np.argmax(prob,axis=0)
    predictionsx.append(y_pred)

    random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
    random_music = random_music[1:]
    
print(predictions)

random music =  [[458 459 257 476  72 141 174 141 419  97 288 303 196 334 297 205  37 101
  190 157  16 325 181 529 279 299 210  60 189 509  46 161]]
random music =  [[459 257 476  72 141 174 141 419  97 288 303 196 334 297 205  37 101 190
  157  16 325 181 529 279 299 210  60 189 509  46 161  18]]
random music =  [[257 476  72 141 174 141 419  97 288 303 196 334 297 205  37 101 190 157
   16 325 181 529 279 299 210  60 189 509  46 161  18  18]]
random music =  [[476  72 141 174 141 419  97 288 303 196 334 297 205  37 101 190 157  16
  325 181 529 279 299 210  60 189 509  46 161  18  18  18]]
random music =  [[ 72 141 174 141 419  97 288 303 196 334 297 205  37 101 190 157  16 325
  181 529 279 299 210  60 189 509  46 161  18  18  18  18]]
random music =  [[141 174 141 419  97 288 303 196 334 297 205  37 101 190 157  16 325 181
  529 279 299 210  60 189 509  46 161  18  18  18  18  18]]
random music =  [[174 141 419  97 288 303 196 334 297 205  37 101 190 157  16 325 181 529
  279 299 

In [85]:
#intergers back to notes
unique_x_int_to_CD = dict((number, note_) for number, note_ in enumerate(unique_x_CD)) 
unique_x_int_to_CD

{0: 'G4.B-5.D6@1.25',
 1: 'F#2.F#3.B-3.D4@0.3',
 2: 'B-3@0.3',
 3: 'C4.E4.G4@0.3',
 4: 'G2.E3.G3.B3.E4@0.3',
 5: 'C#3.G#3.B3.B4@1.25',
 6: 'C4.G#4@0.3',
 7: 'C#4.G5@0.1',
 8: 'E-2.B-2.G3.C4@1.25',
 9: 'A4.F#5@1.25',
 10: 'E-4.G4.B-4.G5@0.3',
 11: 'C#4@1.25',
 12: 'G#4.B4.F5@0.3',
 13: 'C#4@0.1',
 14: 'G3.B-3.B-4@1.25',
 15: 'B-4.E-5.F#5.B-5@0.1',
 16: 'F2.F#2.C#3.F3.C#4@0.1',
 17: 'A3.A5@1.25',
 18: 'F3.C4.C5.F5.A5.C6@0.1',
 19: 'C#3.F#3.A3.C4.E-4.F#4.A4@1.25',
 20: 'G2.E3.G3.D4.G4.D5@0.3',
 21: 'C#4.E4.G4@0.1',
 22: 'E-5.F#5.E-6@0.3',
 23: 'F3.G#3.C#4.C#5@1.25',
 24: 'C4.G5.F6@0.3',
 25: 'E4.B4.B5@0.3',
 26: 'E-4.E-5@0.3',
 27: 'F3.A5.F6@0.3',
 28: 'E3.B4.C#5.D5.E5@0.3',
 29: 'E4.C#5@1.25',
 30: 'B-2.B-4@1.25',
 31: 'F2.E-3.A3.F4@1.25',
 32: 'C3.G#3.F4.B4@0.3',
 33: 'F3@1.25',
 34: 'C3.E-3.C5@0.3',
 35: 'F#5@0.3',
 36: 'E-3@0.3',
 37: 'G#1.G#2.E-3.F#3.C#4@1.25',
 38: 'F#2.F#3@0.3',
 39: 'C4.F4.E-5.A5@1.25',
 40: 'G#4.A4.C#5.G#5.A5@0.3',
 41: 'B2.F#3.D4.D5@1.25',
 42: 'G2.E3.G3.G4.B4.D

In [86]:
predicted_CD = [unique_x_int_to_CD[i] for i in predictions]
predicted_CD

['C4.E4.G4@0.3',
 'C4.E4.G4@0.3',
 'C4.E4.G4@0.3',
 'C4.E4.G4@0.3',
 'C4.E4.G4@0.3',
 'C4.E4.G4@0.3',
 'C4.E4.G4@0.3',
 'C4.E4.G4@0.3',
 'C4.E4.G4@0.3',
 'C4.E4.G4@0.3']

In [94]:
predicted_CD_split = []
for each_outcome in predicted_CD:
    temp_list = []
    temp_list = each_outcome.split("@")
    temp_list[1] =float(temp_list[1])
    predicted_CD_split.append(tuple(temp_list))
    

predicted_CD_split

[('C4.E4.G4', 0.3),
 ('C4.E4.G4', 0.3),
 ('C4.E4.G4', 0.3),
 ('C4.E4.G4', 0.3),
 ('C4.E4.G4', 0.3),
 ('C4.E4.G4', 0.3),
 ('C4.E4.G4', 0.3),
 ('C4.E4.G4', 0.3),
 ('C4.E4.G4', 0.3),
 ('C4.E4.G4', 0.3)]

In [97]:
print("Generating music from our processed chords...")
proccessed_chords_to_midi_sample = get_music_midi_from_chords_and_duration(predicted_CD_split)
print(proccessed_chords_to_midi_sample)
play_midi_file(proccessed_chords_to_midi_sample)

Generating music from our processed chords...
output-20230129-223106.mid
